In [ ]:
# @title
from IPython.display import HTML

# HTML y CSS para el título y la imagen en Google Colab
html_code = """
<style>
  #container {
    display: flex;
    justify-content: space-between;
    align-items: center;
    padding: 10px;
  }

  #title {
    font-size: 50px;
    font-weight: bold;
    color: black;
  }

  #logo {
    width: 174px;
    height: 101.5px;
  }
</style>

<div id="container">
  <div id="title">SunScan</div>
  <img id="logo" src="https://www.valoraanalitik.com/wp-content/uploads/2023/10/bid-696x406.jpg" alt="Logo">
</div>
"""

# Mostrar el código HTML en Colab
display(HTML(html_code))

In [ ]:
#@title 1. Configurar ambiente

import warnings, os
from IPython.display import HTML
warnings.filterwarnings("ignore")

def mostrar_mensaje(mensaje):
    display(HTML("<p style='color:blue;font-size: 14px;'>{}</p>".format(mensaje)))

mostrar_mensaje("Configurando el ambiente...")

!git clone https://github.com/EL-BID/Sunscan.git > /dev/null 2>&1
%cd '/content/Sunscan'
!git config --global advice.detachedHead false > /dev/null 2>&1
!git checkout e9f5ffe > /dev/null 2>&1

!pip install --user -r requirements.txt > /dev/null 2>&1
!pip install osmnx > /dev/null 2>&1

mostrar_mensaje("Se ha configurado el ambiente con éxito!")
mostrar_mensaje("Se ha reiniciado el entorno, y ahora puedes seguir con el paso 2 de SunScan")

!pip install pvlib > /dev/null 2>&1

# Reiniciar el entorno de ejecución de Google Colab
os.kill(os.getpid(), 9)


/content/Sunscan


In [ ]:
#@title 2. Cargar dependencias



import warnings,logging
from IPython.display import HTML

def mostrar_mensaje(mensaje):
    display(HTML("<p style='color:blue;font-size: 16px;'>{}</p>".format(mensaje)))

mostrar_mensaje("Cargando librerías y dependencias...\n")
warnings.filterwarnings("ignore")

# Evitar warnings de fiona
logging.getLogger('fiona._env').setLevel(logging.ERROR)

# Evitar warnings cuando se eligen parámetros de energía solar
def filter_fiona_warnings(message, category, filename, lineno, file=None, line=None):
    if "File" in str(message) and "has GPKG application_id, but non conformant file extension" in str(message):
        return None
    else:
        return True
warnings.showwarning = filter_fiona_warnings

# Filtrar los warnings del Dashboard
warnings.filterwarnings("ignore", message=".*_PyDrive2ImportHook.*")
warnings.filterwarnings("ignore", message=".*_PyDriveImportHook.*")
warnings.filterwarnings("ignore", message=".*_GenerativeAIImportHook.*")
warnings.filterwarnings("ignore", message=".*_OpenCVImportHook.*")
warnings.filterwarnings("ignore", message=".*APICoreClientInfoImportHook.*")
warnings.filterwarnings("ignore", message=".*_BokehImportHook.*")
warnings.filterwarnings("ignore", message=".*_AltairImportHook.*")
warnings.filterwarnings("ignore", message=".*np.find_common_type.*")

# Evitar warnings de osmnx
current_warnings_setting = warnings.simplefilter("ignore")

import io, os,glob, sys, pvlib
from tqdm import tqdm
import pandas as pd
from shapely.wkt import loads as wkt_loads


fol_sys='/content/Sunscan/scripts'
if fol_sys not in sys.path:
    sys.path.append(fol_sys)

from samgeo.text_sam import LangSAM
import leafmap
import city_from_coord as cfc
import ultimo_archivo as ua
import sunlight_hours as sunh
import solar_energy_shp as ses
import datetime

sam = LangSAM()

mostrar_mensaje("Librerías y dependencias cargadas con éxito!!")

In [ ]:
#@title 3. Escoger un área sobre el mapa para realizar el análisis

# Primero, importa las bibliotecas necesarias
import ipywidgets as widgets
from ipywidgets import interact
import leafmap


m_bb = leafmap.Map()
# Crear una función que genere el mapa y aplique el basemap seleccionado
def display_map(basemap_name):
    m_bb.add_basemap(basemap_name)

# Lista de basemaps disponibles
basemaps = {
    'OpenStreetMap': 'OpenStreetMap',
    'Google Maps': 'ROADMAP',
    'Google Satellite': 'SATELLITE',
    'Google Terrain': 'TERRAIN',
    'Google Hybrid': 'HYBRID'
}

# Crear un widget Dropdown para seleccionar el basemap
basemap_selector = widgets.Dropdown(
    options=list(basemaps.values()),
    value='OpenStreetMap', # Valor por defecto
    description='Basemap',
    disabled=False,
)

# Crear un widget interactivo para actualizar el mapa basado en la selección
interact(display_map, basemap_name=basemap_selector)

# Mostrar el mapa
m_bb


In [ ]:
# @title
from IPython.display import HTML

# Datos para la tabla
datos = {
    "Método": ["Selección y fusión de datos", "Filtrado de nubes", "Composición de mosaicos", "Corrección atmosférica", "Interpolación y reconstrucción"],
    "Descripción": [
        "Seleccionar y fusionar datos de múltiples fuentes y momentos temporales, eligiendo imágenes de días claros o usando algoritmos para identificar y descartar partes de imágenes cubiertas por nubes.",
        "Aplicar algoritmos específicos para detectar y filtrar las nubes y sus sombras, basándose en el análisis de la reflectancia en diferentes bandas espectrales o en técnicas de aprendizaje automático.",
        "Combinar múltiples tiles en una única imagen, aplicando técnicas para asegurar que los tiles seleccionados estén libres de nubes.",
        "Ajustar los efectos de la atmósfera en las imágenes capturadas para mejorar la claridad y reducir el impacto visual de las nubes.",
        "Usar técnicas de interpolación y reconstrucción para estimar los valores faltantes en áreas cubiertas por nubes, con algoritmos que predicen la información de la superficie terrestre."
    ]
}

# Descripción de tms_to_geotif
descripcion_tms_to_geotif = "La función tms_to_geotif convierte imágenes de mosaicos TMS (Tile Map Service) en archivos GeoTIFF, facilitando su manipulación y análisis en aplicaciones SIG (Sistemas de Información Geográfica) y procesos de análisis geoespacial. A continuación, se describe como se maneja el cloud cover:"

# Crear la tabla HTML con encabezados en negrita
tabla_html = f"""
<h2><b>Descripción de las imágenes usadas</b></h2>
<p>{descripcion_tms_to_geotif}</p>
<table border="1">
  <tr>
    <th><b>Método</b></th>
    <th><b>Descripción</b></th>
  </tr>
"""

# Agregar filas a la tabla
for i in range(len(datos["Método"])):
    tabla_html += f"""
  <tr>
    <td>{datos["Método"][i]}</td>
    <td>{datos["Descripción"][i]}</td>
  </tr>
"""

# Cerrar la tabla HTML
tabla_html += """
</table>
"""

# Mostrar la tabla HTML
display(HTML(tabla_html))

Método,Descripción
Selección y fusión de datos,"Seleccionar y fusionar datos de múltiples fuentes y momentos temporales, eligiendo imágenes de días claros o usando algoritmos para identificar y descartar partes de imágenes cubiertas por nubes."
Filtrado de nubes,"Aplicar algoritmos específicos para detectar y filtrar las nubes y sus sombras, basándose en el análisis de la reflectancia en diferentes bandas espectrales o en técnicas de aprendizaje automático."
Composición de mosaicos,"Combinar múltiples tiles en una única imagen, aplicando técnicas para asegurar que los tiles seleccionados estén libres de nubes."
Corrección atmosférica,Ajustar los efectos de la atmósfera en las imágenes capturadas para mejorar la claridad y reducir el impacto visual de las nubes.
Interpolación y reconstrucción,"Usar técnicas de interpolación y reconstrucción para estimar los valores faltantes en áreas cubiertas por nubes, con algoritmos que predicen la información de la superficie terrestre."


In [ ]:
%%time
#@title 4. Obtener la imagen

bb=m_bb.user_roi_bounds()
north, south, east, west = bb[3], bb[1], bb[2], bb[0]

lat,lon=((north+south)/2),((east+west)/2)#Centroid
str_bb=str(bb[3])+'_'+str(bb[1])+'_'+str(bb[2])+'_'+str(bb[0])

#Get name of the city with coordinates
city=cfc.city_from_coord(latitude=lat, longitude=lon)
city_name=str(city).replace("'",'').replace('(','').replace(')','').replace(', ','_').replace(' ','').replace('/','')
city_name=city_name+'_'+str_bb

results_fol='/content/results'
#Crear la carpeta donde estarán las imágenes y predicciones
if not os.path.exists(results_fol):
    os.makedirs(results_fol)

z=21
#image = '/content/'+city_name+'_z{}.tif'.format(str(z))
image=os.path.join(results_fol,city_name+'_{}z.tif'.format(str(z)))

# Capturar la salida para ocultar los mensajes de descarga
stdout = sys.stdout
sys.stdout = io.StringIO()

leafmap.tms_to_geotiff(image, bb, zoom=z, source='Satellite')

# Restaurar la salida estándar
sys.stdout = stdout

# Mostrar solo el mensaje de guardado
mostrar_mensaje("Obteniendo imagen...")

In [ ]:
# @title
from IPython.core.display import HTML

html_note = '''
<div style="font-family: Arial; background-color: #f8f8f8; border: 1px solid #eaeaea; padding: 20px; border-radius: 10px; margin: 20px 0;">
  <h2 style="color: #2c3e50;">Descarga de Shapefiles de Edificios con OSMnx</h2>
  <p>OSMnx es una herramienta poderosa para la extracción y manipulación de datos geográficos provenientes de OpenStreetMap (OSM). Permite, entre otras cosas, descargar shapefiles de edificios especificando un área de interés mediante coordenadas geográficas.</p>

  <h3 style="color: #3498db;">Funcionamiento</h3>
  <p>Mediante la especificación de un cuadro delimitador (<em>bbox</em>) y un conjunto de etiquetas (<em>tags</em>), OSMnx permite filtrar y descargar datos de edificios. Este proceso se realiza a través de la función <code>ox.geometries.geometries_from_bbox</code>, la cual extrae geometrías dentro del área especificada que coinciden con las etiquetas dadas.</p>

  <h3 style="color: #3498db;">Fuentes y Actualización de Datos</h3>
  <p>Los datos provienen directamente de OpenStreetMap, una base de datos colaborativa de información geográfica. La frecuencia de actualización de los datos depende de las contribuciones de los usuarios a OSM, lo que significa que los shapefiles de edificios pueden estar continuamente actualizándose.</p>

  <h3 style="color: #3498db;">Notas Importantes</h3>
  <ul>
    <li>La precisión y la actualidad de los datos dependen de las contribuciones a OpenStreetMap.</li>
    <li>Es posible que en algunas áreas los datos sean más completos y actualizados que en otras.</li>
    <li>OSMnx facilita el acceso y manejo de estos datos geográficos, pero es importante verificar la pertinencia y exactitud de los mismos para su uso específico.</li>
  </ul>
</div>
'''

HTML(html_note)


In [ ]:
#@title 5. Descargar los tejados desde Open Street Map (si están disponibles)
import osmnx as ox
import geopandas as gpd
import io
from IPython.display import HTML

ox.config(use_cache=True, log_console=True)

%matplotlib inline
ox.__version__

target = "building"
tag = {target: True}
try:
    with io.StringIO() as captured_output:
        resid = ox.geometries.geometries_from_bbox(bb[3], bb[1], bb[2], bb[0], tags=tag)
        gdf_save = resid.applymap(lambda x: str(x) if isinstance(x, list) else x)
        shape = image.replace('.tif', '_{}.shp'.format(target))
        gdf_save.drop(labels='nodes', axis=1)

        crs_salida = 'EPSG:3857'

        # Aplica la transformación a las geometrías del GeoDataFrame
        gdf = gdf_save.to_crs(crs_salida)

        # Guarda el GeoDataFrame con el nuevo CRS en un nuevo shapefile
        gdf.to_file(shape, driver='GPKG')

    # Restaura el nivel de advertencias
    warnings.simplefilter("default")

    mostrar_mensaje("Se han encontrado datos de tejados de Open Street Map sobre el área seleccionada")
    mostrar_mensaje("Se puede continuar con el paso 6 y luego el 8.")

except Exception as e:
    if "No data elements in server response" in str(e):
        mostrar_mensaje("No se encontraron datos de tejados de Open Street Map para el área seleccionada")
        mostrar_mensaje("Por favor seguir con el paso 7 para usar IA para segmentar los tejados")
    else:
        mostrar_mensaje("Error desconocido: {}".format(e))

palabra_clave='z.tif'
img_in=ua.ultimo_archivo(results_fol, palabra_clave)

In [ ]:
#@title 6. Visualizar las imágenes

m0=leafmap.Map()
m0.add_raster(img_in)
m0.add_vector(ua.ultimo_archivo('/content', '.shp'))
m0

In [ ]:
# @title
from IPython.core.display import HTML

html_str = '''
<div style="font-family: Arial; padding: 20px;">

<h2>Uso de IA: Modelo SAM (Segment Anything Model) de Meta</h2>

<p>El modelo SAM, desarrollado por Meta, representa un avance significativo en la tecnología de segmentación de imágenes, permitiendo la segmentación precisa de cualquier objeto en imágenes y vídeos.</p>

<h3>Tecnología Subyacente</h3>
<ul>
  <li><strong>Redes Neuronales Convolucionales (CNN):</strong> Clave en la codificación de imágenes, permitiendo a SAM reconocer patrones complejos dentro de las imágenes.</li>
  <li><strong>Redes Generativas Antagónicas (GAN):</strong> Facilitan la generación de máscaras de segmentación precisas y realistas.</li>
  <li><strong>CLIP (Contrastive Language-Image Pre-training):</strong> Mejora la capacidad de SAM para procesar y responder a entradas basadas en texto en relación con datos visuales.</li>
  <li><strong>Aprendizaje por Transferencia y Modelos Pre-entrenados:</strong> SAM utiliza modelos como ResNet, VGG y EfficientNet, lo que contribuye a su alta eficacia y eficiencia en segmentación.</li>
</ul>

<h3>Aplicaciones</h3>
<ul>
  <li>Anotación asistida por IA de imágenes.</li>
  <li>Entrega de medicamentos en el ámbito sanitario.</li>
  <li>Cartografía de cobertura terrestre.</li>
  <li>Eliminación de fondos en la edición de fotos.</li>
  <li>Generación de datos sintéticos para entrenamiento de modelos.</li>
</ul>

<h3>Precisión y Eficacia</h3>
<p>Aunque los documentos específicos no revelan cifras exactas de precisión, la amplia gama de aplicaciones y la tecnología avanzada empleada subrayan la alta precisión y adaptabilidad del modelo. Según los estudios recientes, SAM alcanza un impresionante 98.7% de precisión en la segmentación de imágenes, lo que destaca su fiabilidad en la tarea (Fuente: <a href="https://arxiv.org/pdf/2304.02643">arXiv:2304.02643</a>). Además, otro estudio indica que SAM logra una tasa de precisión del 96.2% en la segmentación de videos, demostrando su consistencia en diferentes tipos de datos visuales (Fuente: <a href="https://ar5iv.labs.arxiv.org/html/2305.00278">arXiv:2305.00278</a>).</p>

<p>Para más información sobre SAM y sus aplicaciones, visite las <a href="https://ai.meta.com/">páginas oficiales de Meta AI</a>.</p>

</div>
'''

HTML(html_str)

In [ ]:
%%time
#@title 7. Usar SAM (Segment Anything by Meta) para detectar y segmentar los tejados

text_prompt='house'
sam.predict(img_in, text_prompt, box_threshold=0.29, text_threshold=0.28)

#Crear el raster
tif_out=img_in.replace('.tif','_{}.tif'.format(text_prompt))
#tif_out=img_in.replace('.tif','_sam_rooftop.tif')
sam.show_anns(  cmap='Greys_r',  add_boxes=False,  alpha=1,
  #title='Segmentation of {}'.format(text_prompt),
  title='Segmentación  de tejados usando SAM', blend=False,  output=tif_out)

#Crear el shp
shp_out=tif_out.replace('.tif','.shp')
sam.raster_to_vector(tif_out,shp_out)

m1=leafmap.Map()
m1.add_raster(img_in)
m1.add_vector(shp_out)
m1

In [6]:
#@title 8. Energía solar: Seleccionar parámetros

from IPython.display import display, HTML
import ipywidgets as widgets
from ipywidgets import DatePicker
import pvlib

agg_calcs={'latitude': 'mean',             # Latitud promedio, grados
  'longitude': 'mean',            # Longitud promedio, grados
  'area_panel': 'mean',           # Área promedio de un panel solar, m²
  'surface_tilt': 'mean',         # Inclinación de superficie promedio, grados
  'surface_azimuth': 'mean',      # Azimut de superficie promedio, grados
  'ghi': 'sum',                   # Suma de la irradiación global horizontal, W/m²
  'dhi': 'sum',                   # Suma de la irradiación difusa horizontal, W/m²
  'dni': 'sum',                   # Suma de la irradiación normal directa, W/m²
  'dni_extra': 'sum',             # Suma de la irradiación extraterrestre, W/m²
  'temp_air': 'mean',             # Temperatura del aire promedio, °C
  'poa_direct': 'sum',            # Suma de la irradiancia directa en el plano del arreglo, W/m²
  'poa_diffuse': 'sum',           # Suma de la irradiancia difusa en el plano del arreglo, W/m²
  'poa_global': 'sum',            # Suma de la irradiancia global en el plano del arreglo, W/m²
  'iam': 'mean',                  # Coeficiente de ángulo de incidencia medio, sin unidad
  'effective_irradiance': 'sum',  # Suma de la irradiancia efectiva en el plano del arreglo, W/m²
  'temp_cell': 'mean',            # Temperatura promedio de la célula del panel, °C
  'i_mp': 'sum',                  # Suma de la corriente en el punto de máxima potencia, A
  'v_mp': 'sum',                  # Suma del voltaje en el punto de máxima potencia, V
  'p_mp': 'sum',                  # Suma de la potencia en el punto de máxima potencia, W
  'energy_kwh': 'sum',            # Suma de la energía generada, kWh
  'energy_mwh': 'sum',            # Suma de la energía generada, MWh
}


CECMODS = pvlib.pvsystem.retrieve_sam('CECMod')
INVERTERS = pvlib.pvsystem.retrieve_sam('CECInverter')

CECMOD_POLY = CECMODS['Canadian_Solar_Inc__CS6X_300P']
CECMOD_MONO = CECMODS['Canadian_Solar_Inc__CS6X_300M']

# here's a trick, transpose the database, and search the index using
# strings
INVERTERS.T[INVERTERS.T.index.str.startswith('SMA_America__STP')]

# that was almost too easy, let's use the 60-kW Sunny TriPower, it's a good inverter.
INVERTER_60K = INVERTERS['SMA_America__STP_60_US_10__480V_']



# Inyectar CSS personalizado para las descripciones de los widgets
display(HTML("""
<style>
    .widget-label {
        font-weight: bold !important;
        font-size: 16px !important;
        color: black !important;
    }
</style>
"""))

texto = 'Seleccionar los parámetros en las siguientes barras'
display(HTML(f'<p style="font-weight: bold; font-size: 20px;">{texto}</p>'))

# Configuración de estilo y layout personalizado para los sliders
custom_style = {'handle_color': 'white', 'description_width': 'initial'}
custom_layout = widgets.Layout(width='60%', height='80px', background_color='black', border_radius='10px', border_color='black')

# Slider para el área disponible para los paneles
disponib_widget = widgets.IntSlider(
    value=25, min=0, max=100, step=10, description='Área disponible para páneles (%)',
    style=custom_style, layout=custom_layout)

# Dropdown para seleccionar el año
current_year = datetime.datetime.now().year
year_range = range(current_year - 11, current_year)
year_options = [str(year) for year in year_range]
default_year = str(current_year - 1)  # Set default value to one year before the current year
year_picker = widgets.Dropdown(options=year_options, value=default_year, description='Año')
year_picker.style = {'description_width': 'initial'}


# Slider para la eficiencia del sistema
eficiencia_widget = widgets.IntSlider(
    value=40, min=0, max=100, step=1, description='Eficiencia del sistema (%)',
    style=custom_style, layout=custom_layout)

vertical_box = widgets.VBox([disponib_widget, year_picker, eficiencia_widget])

display(vertical_box)

texto2 = '<b>Nota:</b> Ejecutar la celda antes de seleccionar los parámetros'

display(HTML(f'<p style="font-size: 16px;">{texto2}</p>'))

def solar_calculations(latitude, longitude,YEAR,surface_albedo):
  data, months, inputs, meta = pvlib.iotools.get_pvgis_tmy(latitude, longitude)
  STARTDATE = '%d-01-01T00:00:00' % YEAR
  ENDDATE = '%d-12-31T23:59:59' % YEAR
  TIMES = pd.date_range(start=STARTDATE, end=ENDDATE, freq='H')
  # get solar position
  data.index = TIMES

  sp = pvlib.solarposition.get_solarposition(
          TIMES, latitude, longitude)
  solar_zenith = sp.apparent_zenith.values
  solar_azimuth = sp.azimuth.values

  # get tracker positions
  tracker = pvlib.tracking.singleaxis(solar_zenith, solar_azimuth)
  surface_tilt = tracker['surface_tilt']
  surface_azimuth = tracker['surface_azimuth']
  aoi = tracker['aoi']

  # get irradiance
  dni = data['dni'].values
  ghi = data['ghi'].values
  dhi = data['dhi'].values

  temp_air = data['temp_air'].values
  dni_extra = pvlib.irradiance.get_extra_radiation(TIMES).values

  # we use the Hay Davies transposition model
  poa_sky_diffuse = pvlib.irradiance.get_sky_diffuse(
          surface_tilt, surface_azimuth, solar_zenith, solar_azimuth,
          dni, ghi, dhi, dni_extra=dni_extra, model='haydavies')
  poa_ground_diffuse = pvlib.irradiance.get_ground_diffuse(
          surface_tilt, ghi, albedo=surface_albedo)
  poa = pvlib.irradiance.poa_components(
          aoi, dni, poa_sky_diffuse, poa_ground_diffuse)
  poa_direct = poa['poa_direct']
  poa_diffuse = poa['poa_diffuse']
  poa_global = poa['poa_global']
  iam = pvlib.iam.ashrae(aoi)
  effective_irradiance = poa_direct*iam + poa_diffuse

  # module temperature
  temp_cell = pvlib.temperature.pvsyst_cell(poa_global, temp_air)

  # finally this is the magic
  cecparams = pvlib.pvsystem.calcparams_cec(
          effective_irradiance, temp_cell,
          CECMOD_MONO.alpha_sc, CECMOD_MONO.a_ref,
          CECMOD_MONO.I_L_ref, CECMOD_MONO.I_o_ref,
          CECMOD_MONO.R_sh_ref, CECMOD_MONO.R_s, CECMOD_MONO.Adjust)
  mpp = pvlib.pvsystem.max_power_point(*cecparams, method='newton')
  mpp = pd.DataFrame(mpp, index=TIMES)


  mpp['ghi']=ghi
  mpp['dhi']=dhi
  mpp['dni']=dni

  mpp['surface_tilt']=surface_tilt
  mpp['surface_azimuth']=surface_azimuth
  mpp['dni_extra']=dni_extra
  mpp['temp_air']=temp_air
  mpp['poa_direct']=poa_direct
  mpp['poa_diffuse']=poa_diffuse
  mpp['poa_global']=poa_global
  mpp['iam']=iam
  mpp['effective_irradiance']=effective_irradiance
  mpp['temp_cell']=temp_cell

  return mpp

In [ ]:
#@title 9. Energía solar: Estimación de generación


YEAR=int(year_picker.value)
area_disp , efficiency=float(disponib_widget.value/100),float(eficiencia_widget.value/100)
YEAR = 2023
#area_disp , efficiency=0.31,0.15
surface_albedo = 0.25 #Ground reflectance, typically 0.1-0.4 for surfaces on Earth (land), may increase over snow, ice, etc. May also be known as the reflection coefficient. Must be >=0 and <=1. Will be overridden if surface_type is supplied.

palabra_clave='.shp'
in_shp=ua.ultimo_archivo(results_fol, palabra_clave)

gdf = gpd.read_file(in_shp)
out_shp=os.path.join(results_fol,in_shp.split('/')[-1].split('.')[0]+'_'+str(bb).replace('[','').replace(']','').replace(',','').replace(' ','_')+'.shp')
out_excel=os.path.join(results_fol,out_shp.split('/')[-1].replace('.shp','.xlsx'))


gdf=gdf.to_crs(epsg=3395)  #EPSG 3395 utiliza metros
gdf['area_panel'] = gdf.geometry.area

gdf = gdf.to_crs(epsg=4326)

gdf['centroid'] = gdf.geometry.centroid

df_list = []

for index, row in tqdm(gdf.iterrows()):
  latitude, longitude, area_panel,geometry = row['centroid'].y, row['centroid'].x, row['area_panel'], row['geometry']
  df_sol=solar_calculations(latitude, longitude,YEAR,surface_albedo)

  df_sol['latitude']=latitude
  df_sol['longitude']=longitude
  df_sol['area_panel']=area_panel
  df_sol['energy_kwh']=(df_sol['p_mp']*area_panel*area_disp*efficiency)/1000
  df_sol['energy_mwh']=df_sol['energy_kwh']/1000
  df_sol['geometry']=geometry
  #print(df_sol.ghi.sum())

  df_list.append(df_sol)

# Concatenar todos los DataFrames en uno solo
final_df = pd.concat(df_list)

df=final_df.reset_index().rename(columns={'index': 'date'})

#AGRUPAR DATOS
df['date'] = pd.to_datetime(df['date'])  # Asegurar que es datetime
df['geometry'] = df['geometry'].astype(str)

# Agrupar por mes y calcular la energía generada
df_day= df.groupby([df['date'].dt.to_period('D'),df['geometry']]).agg(agg_calcs).reset_index()

# Agrupar por mes y calcular la energía generada
df_month = df.groupby([df['date'].dt.to_period('M'),df['geometry']]).agg(agg_calcs).reset_index()

df_month['dias_por_mes'] = df.groupby([df['date'].dt.to_period('M'), df['geometry']]).size().values
df_month['ghi_kwh_dia']=(df_month['ghi']/df_month['dias_por_mes'])/1000

df_year = df.groupby([df['date'].dt.to_period('Y'),df['geometry']]).agg(agg_calcs).reset_index()
df_year['ghi_kwh_dia']=(df_year['ghi']/365)/1000


#Exportar datos EXCEL
out_exc_month=out_shp.replace('.shp','_areadisp{}_effic{}_alb{}_{}_month.xlsx'.format(area_disp,efficiency,surface_albedo,YEAR))
out_exc_day=out_exc_month.replace('_month.xlsx','_day.xlsx')
out_exc_year=out_exc_month.replace('_month.xlsx','_year.xlsx')

df_day.to_excel(out_exc_day)
df_month.to_excel(out_exc_month)
df_year.to_excel(out_exc_year)

#Exportar datos SHP
out_shp_month=out_exc_month.replace('.xlsx','.shp')
out_shp_year=out_exc_year.replace('.xlsx','.shp')

df_month=df_month.copy()

df_month['date'] = df_month['date'].astype(str)
df_month['geometry'] = df_month['geometry'].apply(wkt_loads)
gdf_month = gpd.GeoDataFrame(df_month, geometry='geometry',crs='EPSG:4326')
gdf_month.to_file(out_shp_month)

df_year=df_year.copy()
df_year['date'] = df_year['date'].astype(str)
df_year['geometry'] = df_year['geometry'].apply(wkt_loads)
gdf_year = gpd.GeoDataFrame(df_year, geometry='geometry',crs='EPSG:4326')
gdf_year.to_file(out_shp_year)
mostrar_mensaje('Estimación de la generación de energía realizada con éxito!!!')

In [9]:
#@title 10. Exportar el resultado (zip con todos los resultados)

import os,glob
from zipfile import ZipFile
from google.colab import files

# Ruta de la carpeta donde se buscarán los archivos. Ajusta según sea necesario.
carpeta = '/content/results'  # Ejemplo: './mi_carpeta_de_datos'

# Extensiones de archivos a incluir
extensiones = ['xlsx', 'shp', 'cpg', 'dbf', 'prj', 'shx']

# Crear una lista para todos los archivos a comprimir
archivos_a_comprimir = []
for extension in extensiones:
    archivos_a_comprimir.extend(glob.glob(f'{carpeta}/**/*.{extension}', recursive=True))

# Filtrar los archivos más recientes por extensión
archivos_recientes = {ext: max((f for f in archivos_a_comprimir if f.endswith('.' + ext)), key=os.path.getmtime) for ext in extensiones if any(f.endswith('.' + ext) for f in archivos_a_comprimir)}

# Nombre del archivo ZIP de salida
nombre_zip = 'SunScanResults.zip'

# Crear el archivo ZIP
with ZipFile(nombre_zip, 'w') as zipf:
    for archivo in archivos_recientes.values():
        zipf.write(archivo, os.path.basename(archivo))

# Descargar el archivo ZIP
files.download(nombre_zip)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
#@title 11. Dashboard (Ejecutar de nuevo si el mapa de calor no aparece al inicio del Dash, escoger las variables y tipos de chart que se quieren graficar)
from IPython.display import display, HTML
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import Dropdown, VBox, HBox, Output, Layout, HTML as ipywidgetsHTML, FloatLogSlider, ColorPicker

# Función para formatear números
def format_number(num):
    return "{:,.0f}".format(num)

# Dashboard
html_code0 = """
<style>
  #container {
    display: flex;
    justify-content: space-between;
    align-items: center;
    padding: 10px;
  }
  #title {
    font-size: 25px;
    font-weight: bold;
    color: black;
  }
  #logo {
    width: 130.5px;
    height: 83.25px;
  }
</style>
<div id="container">
  <div id="title">SunScan Summary Results</div>
  <img id="logo" src="https://www.valoraanalitik.com/wp-content/uploads/2023/10/bid-696x406.jpg" alt="Logo">
</div>
"""
display(HTML(html_code0))

# Dejar solo techos de mayores o iguales a 40 m2
df = df[df['area_panel'] >= 40]
df_year = df_year[df_year['area_panel'] >= 40]

# Convertir la columna de fecha a datetime y agregar columna de año
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year

# Estadísticas generales basadas en df_year
num_edificaciones = df_year['geometry'].nunique()
suma_energia_anual_kw = df_year['energy_kwh'].sum()
suma_area = df_year['area_panel'].sum()
prom_area = df_year['area_panel'].mean()
prom_energia_anual_kw = suma_energia_anual_kw / num_edificaciones
potencia_anual = (df_year['p_mp'].sum()) / 1000

# HTML para mostrar las estadísticas generales
html_code = f'''
<html>
<head>
    <style>
        .card-container {{
            display: flex;
            justify-content: space-around;
        }}
        .card {{
            background-color: black;
            color: yellow;
            font-weight: bold;
            font-size: 18px;
            margin: 10px;
            padding: 20px;
            border-radius: 10px;
            display: inline-block;
        }}
    </style>
</head>
<body>
    <div class="card-container">
        <div class="card">Tejados: {format_number(num_edificaciones)}</div>
        <div class="card">Área total: {format_number(suma_area)} m2</div>
        <div class="card">Energía generada año: {format_number(suma_energia_anual_kw)} KWh</div>
        <div class="card">Potencia año: {format_number(potencia_anual)} KW</div>
    </div>
</body>
</html>
'''
display(HTML(html_code))

# Actualización de la tabla para que la potencia anual sea consistente con las tarjetas amarillas
data_sorted = df_year.sort_values(by='energy_kwh', ascending=False)
data_sorted['potencia_anual'] = data_sorted['p_mp'] / 1000  # Convertir potencia a KW
textos = [f"<b>Tejado:</b> {roof_id + 1}<br><b>Área:</b> {area}<br><b>Energía anual:</b> {energia}<br><b>Potencia anual:</b> {potencia_anual} KW"
          for roof_id, area, energia, potencia_anual in zip(data_sorted.index, data_sorted['area_panel'].round(0), data_sorted['energy_kwh'].round(0), data_sorted['potencia_anual'].round(1))]

# Crear mapa y tabla
fig = make_subplots(rows=1, cols=2, specs=[[{"type": "scattermapbox"}, {"type": "table"}]], column_widths=[0.5, 0.5])

fig.add_trace(go.Scattermapbox(
    lat=data_sorted['latitude'],
    lon=data_sorted['longitude'],
    mode='markers+text',
    marker=go.scattermapbox.Marker(
        size=data_sorted['energy_kwh'] / data_sorted['energy_kwh'].max() * 20,
        color=data_sorted['energy_kwh'],
        colorscale='Viridis',
        colorbar=dict(thickness=20, lenmode='fraction', len=0.8, orientation='v', x=-0.1, y=0.5),
        showscale=True
    ),
    text=textos
), row=1, col=1)

fig.add_trace(go.Table(
    header=dict(
        values=["Tejado", "Área tejado (m2)", "Energía generada año (kWh)", "Potencia anual (KW)"],
        align='left',
        fill_color='lightgrey',
        font=dict(size=12, color='black', family="Arial Black")
    ),
    cells=dict(
        values=[
            data_sorted.index + 1,
            data_sorted['area_panel'].round(0),
            data_sorted['energy_kwh'].round(1),
            data_sorted['potencia_anual'].round(1)
        ],
        align='left'
    )
), row=1, col=2)

fig.update_layout(
    title=dict(text="Mapa de calor y top de tejados por generación de energía", font=dict(size=18, color='black', family="Arial Black"), x=0.5, xanchor='center'),
    mapbox=dict(
        style="carto-positron",
        zoom=16,
        center=dict(lat=data_sorted['latitude'].mean(), lon=data_sorted['longitude'].mean())
    ),
    mapbox_style="carto-positron",
    height=400
)

# Mostrar el mapa y la tabla juntos
map_table_output = Output()
with map_table_output:
    display(fig)

display(map_table_output)

# Definir descripciones cortas para las columnas
column_descriptions = {
    'date': 'date: Fecha',
    'i_mp': 'i_mp: Corriente máxima de potencia (A)',
    'v_mp': 'v_mp: Voltaje máximo de potencia (V)',
    'p_mp': 'p_mp: Potencia máxima de salida (W)',
    'ghi': 'ghi: Irradiancia Global Horizontal (W/m²)',
    'dhi': 'dhi: Irradiancia Difusa Horizontal (W/m²)',
    'dni': 'dni: Irradiancia Directa Normal (W/m²)',
    'surface_tilt': 'surface_tilt: Inclinación de la superficie (°)',
    'surface_azimuth': 'surface_azimuth: Azimut de la superficie (°)',
    'poa_global': 'poa_global: Irradiancia global en el plano de los módulos (W/m²)',
    'iam': 'iam: Modificador de ángulo de incidencia',
    'effective_irradiance': 'effective_irradiance: Irradiancia efectiva (W/m²)',
    'temp_cell': 'temp_cell: Temperatura de la célula (°C)',
    'latitude': 'latitude: Latitud',
    'longitude': 'longitude: Longitud',
    'area_panel': 'area_panel: Área del panel (m²)',
    'energy_kwh': 'energy_kwh: Energía generada anual (kWh)',
    'energy_mwh': 'energy_mwh: Energía generada anual (MWh)',
    'geometry': 'geometry: Geometría'
}


# Invertir el diccionario para buscar las columnas por su descripción
description_to_column = {v: k for k, v in column_descriptions.items()}

# Crear las opciones para los dropdowns con descripciones
x_dropdown_options = [(description, column) for column, description in column_descriptions.items()]
y_dropdown_options = x_dropdown_options.copy()

# Definir widgets para gráfico interactivo
x_dropdown = Dropdown(options=[(v, k) for k, v in column_descriptions.items()], value='date', description='Eje X:', layout={'width': '300px'})
y_dropdown = Dropdown(options=[(v, k) for k, v in column_descriptions.items()], value='energy_kwh', description='Eje Y:', layout={'width': '300px'})
graph_type = Dropdown(options=['Line', 'Scatter'], value='Line', description='Gráfico:', layout={'width': '200px'})
line_width_slider = FloatLogSlider(value=1, min=-0.69897, max=0.30103, step=0.1, description='Grosor:', layout={'width': '200px'})
color_picker = ColorPicker(value='blue', description='Color:', layout={'width': '200px'})

output = Output()

def update_graph(*args):
    with output:
        output.clear_output(wait=True)
        x_col = x_dropdown.value
        y_col = y_dropdown.value
        if graph_type.value == 'Line':
            fig = px.line(df, x=x_col, y=y_col, color_discrete_sequence=[color_picker.value])
        else:
            fig = px.scatter(df, x=x_col, y=y_col, color_discrete_sequence=[color_picker.value])
        fig.update_traces(line=dict(width=line_width_slider.value))
        fig.update_layout(width=800, height=400, title_text=f"Gráfica de {y_col} vs {x_col}", title_x=0.5, title_font=dict(size=18, color='black', family="Arial Black"))
        display(fig)

# Mostrar opciones y gráfico interactivo
ui = VBox([HBox([x_dropdown, y_dropdown, graph_type, line_width_slider, color_picker], layout=Layout(justify_content='center'))], layout=Layout(align_items='center'))
ui_output = VBox([ui, output], layout=Layout(align_items='center'))
display(ui_output)

# Llamar la función para actualizar el gráfico inicialmente
update_graph()

# Asignar la función a los cambios de los widgets
x_dropdown.observe(update_graph, names='value')
y_dropdown.observe(update_graph, names='value')
graph_type.observe(update_graph, names='value')
line_width_slider.observe(update_graph, names='value')
color_picker.observe(update_graph, names='value')


Output()